#### Embeddings Model Evaluation

In this notebook we aim to evaluate the performance of difference HuggingFace Embedding models on the nfcorpus benchmark in order to decide with one would be best suited for use in our pipeline

In [1]:
%pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.1
    Uninstalling pydantic_core-2.27.1:
      Successfully uninstalled pydantic_core-2.27.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.2
    Uninstalling pydantic-2.10.2:
      Successfully uninstalled pydantic-2.10.2


In [2]:
%pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.7 MB/s eta 0:00:00


In [3]:
%pip install beir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63550 sha256=02705970ea6f4fbed3a953baccad13363116fce19dd24d01c4df71fee80f9d26
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a6183af09d389459195ff5f854862
  Created wheel for p

In [6]:
import io
import sys
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.base.base_retriever import BaseRetriever
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import Document
from llama_index.core.evaluation.benchmarks import BeirEvaluator

def create_retriever(documents, model_name):
    embed_model = HuggingFaceEmbedding(model_name=model_name)
    index = VectorStoreIndex.from_documents(
        documents, embed_model=embed_model, show_progress=True
    )
    return index.as_retriever(similarity_top_k=30)

models_to_test = [
    "BAAI/bge-small-en-v1.5",
    "BAAI/bge-large-en",
    "sentence-transformers/all-mpnet-base-v2",
    "intfloat/e5-large",
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/gtr-t5-large",
    "BAAI/bge-large-zh-v1.5"
]

results = []

for model_name in models_to_test:
    print(f"Testing model: {model_name}")

    retriever_function = lambda docs: create_retriever(docs, model_name)

    BeirEvaluator().run(
        retriever_function,
        datasets=["nfcorpus"],
        metrics_k_values=[3, 10, 30]
    )


Testing model: BAAI/bge-small-en-v1.5
Dataset: nfcorpus downloaded at: /tmp/llama_index/datasets/BeIR__nfcorpus
Evaluating on dataset: nfcorpus
-------------------------------------


  0%|          | 0/3633 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/3633 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1595 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1595 [00:00<?, ?it/s]

Retriever created for:  nfcorpus
Evaluating retriever on questions against qrels


100%|██████████| 323/323 [01:18<00:00,  4.14it/s]

Results for: nfcorpus
{'NDCG@3': 0.26704, 'MAP@3': 0.05119, 'Recall@3': 0.06084, 'precision@3': 0.24665}
{'NDCG@10': 0.22158, 'MAP@10': 0.06966, 'Recall@10': 0.10174, 'precision@10': 0.16223}
{'NDCG@30': 0.19627, 'MAP@30': 0.07918, 'Recall@30': 0.14605, 'precision@30': 0.09649}
-------------------------------------


Higher is better for all the evaluation metrics.

This [towardsdatascience article](https://towardsdatascience.com/ranking-evaluation-metrics-for-recommender-systems-263d0a66ef54) covers NDCG, MAP and MRR in greater depth.

| **Modèle**                       | **NDCG@3** | **MAP@3** | **Recall@3** | **Precision@3** | **NDCG@10** | **MAP@10** | **Recall@10** | **Precision@10** | **NDCG@30** | **MAP@30** | **Recall@30** | **Precision@30** |
|-----------------------------------|------------|-----------|--------------|-----------------|-------------|------------|---------------|------------------|-------------|------------|---------------|-------------------|
| BAAI/bge-small-en-v1.5            | 0.38766    | 0.0847    | 0.09833      | 0.37255         | 0.3361      | 0.11749    | 0.15986       | 0.25325          | 0.30161     | 0.13719    | 0.22052       | 0.15294           |
| BAAI/bge-large-en                 | 0.39825    | 0.09866   | 0.10807      | 0.37564         | 0.34273     | 0.13283    | 0.16902       | 0.25077          | 0.31692     | 0.15491    | 0.23701       | 0.15542           |
| sentence-transformers/all-mpnet-base-v2 | 0.38528 | 0.08685   | 0.09944      | 0.37049         | 0.33366     | 0.12104    | 0.16199       | 0.25201          | 0.30722     | 0.1425     | 0.23177       | 0.15624           |
| intfloat/e5-large                 | **0.42656**| **0.10532**| **0.11589**  | **0.40248**     | **0.37556** | **0.14489**| **0.18625**   | **0.27709**      | **0.3455**  | **0.16929**| **0.25686**   | **0.17069**       |
| sentence-transformers/all-MiniLM-L6-v2 | 0.35476 | 0.07489   | 0.08583      | 0.33746         | 0.31425     | 0.11007    | 0.15886       | 0.24025          | 0.28666     | 0.12801    | 0.21727       | 0.14737           |
| sentence-transformers/gtr-t5-large| 0.39043    | 0.09262   | 0.10317      | 0.36739         | 0.32691     | 0.12083    | 0.15619       | 0.23406          | 0.29553     | 0.13804    | 0.21242       | 0.1418            |
| BAAI/bge-large-zh-v1.5            | 0.26704    | 0.05119   | 0.06084      | 0.24665         | 0.22158     | 0.06966    | 0.10174       | 0.16223          | 0.19627     | 0.07918    | 0.14605       | 0.09649           |

After evaluation it thus appears that the `intflot/e5-large` model is the best choice